In [2]:
import pandas as pd
import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# --- 準備：ユーザー入力の前処理用関数 ---
def preprocess_query(text, tagger, stop_words):
    """ユーザーの入力文を、分析で使うBi-gramの文字列に変換する"""
    words = []
    for line in tagger.parse(text).splitlines():
        if line == 'EOS' or line == '': continue
        parts = line.split('\t')
        if len(parts) == 2:
            if parts[1].split(',')[0] not in ['助詞', '助動詞', '接続詞', '記号']:
                words.append(parts[0])

    filtered_words = [word for word in words if word not in stop_words]
    bigrams = [f"{filtered_words[i]} {filtered_words[i+1]}" for i in range(len(filtered_words) - 1)]
    return ' '.join(bigrams)

# --- メイン処理 ---
if __name__ == '__main__':
    # 1. 「掃除済み」のCSVファイルを読み込む
    try:
        df = pd.read_csv('/Users/dangararara/lecture/miraisouzou/csv/odaiba_reviews_cleaned.csv')
    except FileNotFoundError:
        print("エラー: 'reviews_cleaned.csv'が見つかりません。")
        exit()

    df.dropna(subset=['cleaned_text'], inplace=True)

    corpus = df['cleaned_text'].tolist()
    shop_names = df['shop_name'].tolist()

    if corpus:
        # 2. お店ごとの特徴ベクトル（TF-IDF）を作成
        print("TF-IDFモデルを学習中...")
        vectorizer = TfidfVectorizer(ngram_range=(2, 2), min_df=2, max_df=0.9)
        tfidf_matrix = vectorizer.fit_transform(corpus)
        print("学習完了。")

        # 3. MeCabとストップワードを準備
        tagger = MeCab.Tagger()
        stop_words = [
            'こと', 'もの', 'それ', 'これ', 'さん', 'よう', 'ため', '的', '気', '感', 'いる', 'する', 
            'なる', 'ある', 'ない', 'くる', 'いく', '思う', '言う', '美味しい', 'うまい', '普通', 
            '良い', '悪い', '最高', '本当', '今回', '前回', '初', '再訪', '人気', '有名', '定番', 
            '満足', '好き', 'おすすめ', 'ごちそうさま', '様', '料理', '食事', 'メニュー', 'ドリンク', 
            'デザート', '一品', 'ご飯', 'ビール', 'ワイン', 'コーヒー', '肉', '魚', '野菜', 
            '食べる', '飲む', '行く', '来る', '入る', '出る', '頼む', '注文', '予約', '訪問', 
            '利用', '頂く', 'いただく', '会計', '時間', '日', '時', '分', 'お店', 'レストラン', 
            'こちら', 'ところ', '場所', '雰囲気', '店内', '外観', '席', 'カウンター', 'テーブル', 
            '個室', 'スタッフ', '店員', '自分', '私', '人', '一つ', 'たくさん', '少し', 'ちょっと', 
            'かなり', '非常', 'いろいろ', '色々', '種類', '豊富', '一番', '円', '...'
        ]

        # 推薦ロジック 
        
        # 4. ユーザーの曖- 曖昧な要望を定義
        user_query = "パンケーキが美味しいお店"
        
        print(f"\nユーザーの要望: 「{user_query}」\n")

        # 5. ユーザーの要望を前処理し、ベクトルに変換
        query_processed = preprocess_query(user_query, tagger, stop_words)
        query_vector = vectorizer.transform([query_processed])

        # 6. コサイン類似度を計算
        cosine_similarities = cosine_similarity(query_vector, tfidf_matrix)
        scores = cosine_similarities[0]
        
        # 7. 結果を整形してランキング表示
        results_df = pd.DataFrame({'shop_name': shop_names, 'score': scores})
        top_10_recommendations = results_df.sort_values(by='score', ascending=False).head(10)

        print("--- 推薦結果 TOP 10 ---")
        print(top_10_recommendations.to_string(index=False))

    else:
        print("分析対象のデータがありません。")

TF-IDFモデルを学習中...
学習完了。

ユーザーの要望: 「パンケーキが美味しいお店」

--- 推薦結果 TOP 10 ---
                              shop_name  score
オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明)    0.0
                                ア・タ・ゴール    0.0
                                ア・タ・ゴール    0.0
                                ア・タ・ゴール    0.0
                                ア・タ・ゴール    0.0
                                ア・タ・ゴール    0.0
                                ア・タ・ゴール    0.0
                                ア・タ・ゴール    0.0
                                ア・タ・ゴール    0.0
                                ア・タ・ゴール    0.0
